In [254]:
import pandas as pd
import os
# os.listdir('../dataset_sheets/pre_pipeline')
df = pd.read_csv('../dataset_sheets/pre_pipeline/labeling/data-hunters-handmarked.csv',on_bad_lines='skip',sep=';')


In [234]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertModel, BertConfig


In [235]:
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification
from torch.nn import BCEWithLogitsLoss

In [240]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [268]:
def add_deleted_name(df): #returns df with new column where data vendor name is deleted from dataset name
    # df = pd.read_csv(path).fillna('')
    df['replaced'] = ''
    for i,row in df.iterrows():
        row['Title'] = row['Title'].replace(row['Data Provider'],'').strip('\'s').strip(': ').strip(' - ').strip(' | ')
    return df


df = df.fillna('')
df = df[df['Data Category']!='']
print(df.columns)
# df = df [['Title', 'Data Category']]
df[['Title', 'Data Category']]
df = add_deleted_name(df)
df[['Title', 'Data Provider']]

Index(['ID', 'Title', 'Content', 'Excerpt', 'Data Provider', 'Use Case',
       'Data Category', 'Related Data sets', 'replaced'],
      dtype='object')


,Title,Data Provider
0,KYC Solution,Datarama
1,Wealth Finder,Datarama
2,Risk Consulting,Datarama
3,STREAM,DataSift
4,Opt-In Email List,Datastream
...,...,...
37079,Greenspace and sudden unexpected death,U.S. EPA Office of Research and Development (ORD)
37080,Arden-Literature Review of Constructed Wetland...,U.S. EPA Office of Research and Development (ORD)
37085,Greenspaces and biomarkers of allostatic load,U.S. EPA Office of Research and Development (ORD)
37086,"LCA of fuels and stoves in India, China, Ghana...",U.S. EPA Office of Research and Development (ORD)


In [269]:
allcats = {}
for i,row in df.iterrows():
    temp = row['Data Category'].split('|')
    # print(temp)
    for q in temp:
        if q in allcats:
            allcats[q] += 1
        else:
            allcats[q] = 1

ac = dict(sorted(allcats.items(), key=lambda item: item[1], reverse=True))
print(ac)
print(len(ac))
# ac = dict(sorted(allcats.items(), key=lambda item: item[1]))


{'Environmental Data': 21255, 'Geospatial Data': 21032, 'Map Data': 7257, 'Demographic Data': 4937, 'Census and Population Data': 3865, 'Economic Data': 3532, 'Construction Industry Data': 1307, 'Soil Health Data': 1196, 'Employment Data': 1071, 'Satellite Data': 787, 'B2B Intent Data': 638, 'Company Data': 582, 'Stock & Market Data': 561, 'Firmographic Data': 404, 'Tourism Data': 382, 'Traffic and Routing Data': 364, 'Energy Data': 326, 'Education Industry Data': 301, 'Real Estate Data': 298, 'Legal Case Data': 289, 'Legal and IP Data': 276, 'Trade Data': 275, 'Public Health Data': 257, 'Business Credit Rating Data': 236, 'Web Data': 232, 'Healthcare Data': 229, 'Marketing Attribution Data': 215, 'Individual Data': 202, 'Agriculture Data': 201, 'B2B Data': 195, 'Business Registry Data': 190, 'Retail & Commerce Data': 148, 'Climate and Weather Data': 142, 'Online/Mobile Data': 134, 'Consumer Lifestyle Data': 134, 'SEO & Web Advertising Data': 132, 'Location Data': 129, 'Industry-Specif

In [273]:
acc = {k:v for k,v in ac.items() if v > 50}
label_list = list(acc.keys())
print(len(acc))
print(label_list)

49
['Environmental Data', 'Geospatial Data', 'Map Data', 'Demographic Data', 'Census and Population Data', 'Economic Data', 'Construction Industry Data', 'Soil Health Data', 'Employment Data', 'Satellite Data', 'B2B Intent Data', 'Company Data', 'Stock & Market Data', 'Firmographic Data', 'Tourism Data', 'Traffic and Routing Data', 'Energy Data', 'Education Industry Data', 'Real Estate Data', 'Legal Case Data', 'Legal and IP Data', 'Trade Data', 'Public Health Data', 'Business Credit Rating Data', 'Web Data', 'Healthcare Data', 'Marketing Attribution Data', 'Individual Data', 'Agriculture Data', 'B2B Data', 'Business Registry Data', 'Retail & Commerce Data', 'Climate and Weather Data', 'Online/Mobile Data', 'Consumer Lifestyle Data', 'SEO & Web Advertising Data', 'Location Data', 'Industry-Specific Data', 'B2B Marketing Fit Data', 'Customer Transaction Data', 'Automotive Industry Data', 'Cross-Device Identity Data', 'Credit Data', 'Identity Data', 'Points of Interest Data', 'Sports & E

In [275]:
def vectorize(s, ac):
    vector = [1 if i in s.split('|') else 0 for i in ac]
    return vector

    
df['List'] = df['Title'].apply(vectorize, ac=acc)
print(df.columns)
print(df['List'][:10])
df_new = df[['Title', 'List']]

Index(['ID', 'Title', 'Content', 'Excerpt', 'Data Provider', 'Use Case',
       'Data Category', 'Related Data sets', 'replaced', 'List'],
      dtype='object')
0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: List, dtype: object


In [277]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [276]:
print(cuda.is_available())

False


In [278]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.Title
        self.targets = self.data.List
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [279]:
train_size = 0.9
train_dataset=df_new.sample(frac=train_size,random_state=200)
test_dataset=df_new.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df_new.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)


FULL Dataset: (33723, 2)
TRAIN Dataset: (30351, 2)
TEST Dataset: (3372, 2)


In [300]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [301]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, len(acc))
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [302]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [303]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [304]:
def train(epoch):
    print(len(training_loader))
    model.train()
    for _,data in enumerate(training_loader, 0):
        print(_)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [305]:
for epoch in range(EPOCHS):
    train(epoch)

3794
0
Epoch: 0, Loss:  0.6810389757156372
1
2
3
4
5


KeyboardInterrupt: 

In [134]:
from tqdm import tqdm_notebook as tqdm

In [27]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join('../labeling/models', "finetuned_pytorch_model.bin")
torch.save(model_to_save.state_dict(), output_model_file)

In [17]:
model_state_dict = torch.load('../labeling/models/finetuned_pytorch_model.bin')

INFERENCE

In [71]:
class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels=2):
        super(BertForMultiLabelSequenceClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            return loss
        else:
            return logits
        
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [12]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

In [18]:
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()
    
    def get_test_examples(self, data_dir, data_file_name, size=-1):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError() 

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

In [19]:
class MultiLabelTextProcessor(DataProcessor):
    
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.labels = None
    
    
    def get_train_examples(self, data_dir, size=-1):
        filename = 'train.csv'
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, filename)))
        if size == -1:
            data_df = pd.read_csv(os.path.join(data_dir, filename))
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df, "train")
        else:
            data_df = pd.read_csv(os.path.join(data_dir, filename))
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df.sample(size), "train")
        
    def get_dev_examples(self, data_dir, size=-1):
        """See base class."""
        filename = 'val.csv'
        if size == -1:
            data_df = pd.read_csv(os.path.join(data_dir, filename))
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df, "dev")
        else:
            data_df = pd.read_csv(os.path.join(data_dir, filename))
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df.sample(size), "dev")

    def get_test_examples(self, data_dir, data_file_name, size=-1):
        data_df = pd.read_csv(os.path.join(data_dir, data_file_name))
#         data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
        if size == -1:
            return self._create_examples(data_df, "test")
        else:
            return self._create_examples(data_df.sample(size), "test")

    def get_labels(self):
        """See base class."""
        if self.labels == None:
            self.labels = list(pd.read_csv(os.path.join(self.data_dir, "classes.txt"),header=None)[0].values)
        return self.labels

    def _create_examples(self, df, set_type, labels_available=True):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, row) in enumerate(df.values):
            guid = row[0]
            text_a = row[1]
            if labels_available:
                labels = row[2:]
            else:
                labels = []
            examples.append(
                InputExample(guid=guid, text_a=text_a, labels=labels))
        return examples

In [20]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

In [21]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [23]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(float(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [82]:
args = {
    "train_size": -1,
    "val_size": -1,
    # "full_data_dir": DATA_PATH,
    # "data_dir": PATH,
    "task_name": "toxic_multilabel",
    "no_cuda": False,
    # "bert_model": BERT_PRETRAINED_PATH,
    # "output_dir": CLAS_DATA_PATH/'output',
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 32,
    "eval_batch_size": 32,
    "learning_rate": 3e-5,
    "num_train_epochs": 4.0,
    "warmup_proportion": 0.1,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": False,
    "loss_scale": 128
}

In [104]:
def predict(model, path, test_filename='test_gov_lt.csv'):
    predict_processor = MultiLabelTextProcessor(path)
    test_examples = predict_processor.get_test_examples(path, test_filename, size=-1)
    
    # Hold input data for returning it 
    input_data = [{ 'id': input_example.guid, 'Title': input_example.text_a } for input_example in test_examples]
    print('IMPUT DATA', input_data)

    test_features = convert_examples_to_features(
        test_examples, label_list, args['max_seq_length'], tokenizer)
    
    logger.info("***** Running prediction *****")
    logger.info("  Num examples = %d", len(test_examples))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    
    # Run prediction for full data
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args['eval_batch_size'])
    
    all_logits = None
    
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(tqdm(test_dataloader, desc="Prediction Iteration")):
        input_ids, input_mask, segment_ids = batch
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)
            logits = logits.sigmoid()

        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    return pd.merge(pd.DataFrame(input_data), pd.DataFrame(all_logits, columns=label_list), left_index=True, right_index=True)

In [ ]:
model = BertForMultiLabelSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(acc), state_dict=model_state_dict)
model.to(device)

In [ ]:
DATA_PATH = '../dataset_sheets/pre_pipeline/labeling/'
result = predict(model, DATA_PATH)

In [132]:
dfff = pd.read_csv(os.path.join(DATA_PATH, 'test_gov_lt.csv'))#.create_index( inplace=True)
a = dfff.head
a
# dfff.columns
len(dfff)

255

In [136]:
result.shape

(255, 43)

In [137]:
cols = acc

In [141]:
result.columns
result.head(5)

,id,Title,Demographic Data,Census and Population Data,Economic Data,Construction Industry Data,Employment Data,B2B Intent Data,Company Data,Stock & Market Data,...,Legal and IP Data,Consumer Lifestyle Data,Customer Transaction Data,Industry-Specific Data,B2B Marketing Fit Data,Cross-Device Identity Data,Automotive Industry Data,Credit Data,Points of Interest Data,Social Data
0,0,List of companies licensed to produce beer,0.616299,0.507772,0.557883,0.476998,0.409134,0.412928,0.412344,0.451411,...,0.430333,0.517916,0.596153,0.499197,0.468454,0.644364,0.446601,0.537777,0.482502,0.526952
1,1,List of companies licensed to produce alcoholi...,0.618152,0.499477,0.560525,0.486208,0.426581,0.428528,0.416714,0.451667,...,0.445067,0.505346,0.600065,0.500034,0.477208,0.645918,0.451796,0.547249,0.477928,0.540403
2,2,List of companies holding licenses for the pro...,0.610954,0.480690,0.573754,0.483164,0.431224,0.426026,0.427970,0.477310,...,0.437029,0.512694,0.578926,0.516095,0.448400,0.643474,0.457748,0.552868,0.447352,0.510053
3,3,Other classifications of JAR attributes,0.596935,0.495317,0.552336,0.454822,0.411303,0.418416,0.426577,0.497369,...,0.447986,0.530654,0.579579,0.510689,0.471135,0.627860,0.450128,0.549728,0.481182,0.500733
4,4,Open design data of the Republic of Lithuania,0.592917,0.480318,0.577382,0.475000,0.404420,0.396464,0.423490,0.459900,...,0.446574,0.507616,0.563264,0.512501,0.446403,0.643013,0.468283,0.566291,0.481887,0.499442


POSTPROCESSING

In [217]:
result_0 = result.iloc[:,:2]
result_0

,id,Title
0,0,List of companies licensed to produce beer
1,1,List of companies licensed to produce alcoholi...
2,2,List of companies holding licenses for the pro...
3,3,Other classifications of JAR attributes
4,4,Open design data of the Republic of Lithuania
...,...,...
250,250,Data from several projects
251,251,Road weather data
252,252,Road network data of national and local signif...
253,253,Financial data on road asset values and deprec...


In [220]:
result_1 = result.iloc[:,2:]
for i in result_1.columns:
    result_1[i] = np.where((result_1[i]<=0.6), 0, result_1[i])
    result_1[i] = np.where((result_1[i]>0.6), i, result_1[i])

result_1 = result_1.replace('0.0','')
result_1['data_categories'] = result_1.values.tolist()

In [221]:

def delete_empty(ll):
    l = []
    for i in ll:
        if i:
            l.append(i)
    return l

result_1['data_categories'] = result_1['data_categories'].apply(delete_empty)
result_1['data_categories'] = result_1['data_categories'].apply(lambda x: ','.join(x))
result_2 = result_1[['data_categories']]

In [227]:
final = pd.merge(result_0, result_2,left_index=True, right_index=True)
final
final.to_csv('../results/pre_pipeline/data.gov.lt-datacat-bert.csv')